In [1]:
# Clive Jake A. Gabotero
# ECE41S1
# COE 005

!pip3 install tensorflow
!pip3 install tensorflow-gpu
!pip3 install keras
!pip3 install tensorboard
!pip3 install tensorflow-estimator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 1.7 MB 47.1 MB/s 
     |████████████████████████████████| 438 kB 57.1 MB/s 
     |████████████████████████████████| 5.9 MB 31.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Fo

In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# re-size all the images to 224x 224 pixels for faster processing
IMAGE_SIZE = [224, 224]

#storing the path where the test and training dataset are located in the drive
train_path = '/content/drive/MyDrive/Covid 19 Images/train'
test_path = '//content/drive/MyDrive/Covid 19 Images/test'



#creating preprocessing layer for the data
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [5]:
#adding layers to the model (using all the possible models)
for layer in vgg.layers:
  layer.trainable = False

#storing the directory of the classifications of the dataset
folders = glob('/content/drive/MyDrive/Covid 19 Images/train/*')
folders

['/content/drive/MyDrive/Covid 19 Images/train/Normal',
 '/content/drive/MyDrive/Covid 19 Images/train/Covid',
 '/content/drive/MyDrive/Covid 19 Images/train/Viral Pneumonia']

In [6]:
# creating layers 
x = Flatten()(vgg.output)

#locating the classifications for the datasets
prediction = Dense(len(folders), activation='softmax')(x)

# creating model
model = Model(inputs=vgg.input, outputs=prediction)


In [7]:
# compiling the model given the parameters
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
# Importing all the training images to a variable

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# training sets data
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)

#test sets data
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 249 images belonging to 3 classes.
Found 65 images belonging to 3 classes.


In [11]:
#model fitting 
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
8/8 [==============================] - 173s 21s/step - loss: 1.5577 - accuracy: 0.5261 - val_loss: 0.6175 - val_accuracy: 0.6923
Epoch 2/10
8/8 [==============================] - 153s 20s/step - loss: 0.6707 - accuracy: 0.7189 - val_loss: 0.3530 - val_accuracy: 0.8308
Epoch 3/10
8/8 [==============================] - 153s 19s/step - loss: 0.3750 - accuracy: 0.8474 - val_loss: 0.3685 - val_accuracy: 0.7846
Epoch 4/10
8/8 [==============================] - 174s 22s/step - loss: 0.2857 - accuracy: 0.8755 - val_loss: 0.2365 - val_accuracy: 0.9231
Epoch 5/10
8/8 [==============================] - 160s 20s/step - loss: 0.2255 - accuracy: 0.9036 - val_loss: 0.2319 - val_accuracy: 0.9077
Epoch 6/10
8/8 [==============================] - 165s 21s/step - loss: 0.1493 - accuracy: 0.9398 - val_loss: 0.2535 - val_accuracy: 0.8769
Epoch 7/10
8/8 [==============================] - 154s 20s/step - loss: 0.0856 - accuracy: 0.9759 - val_loss: 0.1525 - val_accuracy: 0.9538
Epoch 8/10
8/8 [====

In [12]:
# save the model to be used for classification later
import tensorflow as tf
from keras.models import load_model

model.save('/content/drive/MyDrive/Covid 19 Images/my_model_vgg16.h5')

In [13]:
#prediction 

#libraries to be use
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('/content/drive/MyDrive/Covid 19 Images/my_model_vgg16.h5')


#creating list of path of the images to be classify
dire=['/content/drive/MyDrive/Covid 19 Images/validation/pic1.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic2.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic3.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic4.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic5.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic6.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic7.jpg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic8.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic9.jpeg',
           '/content/drive/MyDrive/Covid 19 Images/validation/pic10.jpeg'
]

#looping to classify all the images 
prednow=[]
predbef=[]
for im in dire:
  img = image.load_img(im,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  img_data=preprocess_input(x)
  classes = model.predict(img_data)
  new=classes[0]
  i=0
  image
  predbef.append(new)
  for element in new:
    if round(element) ==1:
      if i==0:
        prednow.append("Covid")
      if i==1:
        prednow.append("Normal")
      if i==2:
        prednow.append("Viral")
    i=i+1

#displaying results of the prediction (pic1 to pic10)
print(prednow)


1/1 [==============================] - 1s 507ms/step
['Normal', 'Normal', 'Normal', 'Viral', 'Normal', 'Normal', 'Viral', 'Viral', 'Viral', 'Normal']
